In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import datetime as dt
pd.options.display.max_columns = 120
# from datetime import timedelta
import glob as glob
from datetime import datetime
import os
import zipfile
import time
import modin.pandas as pd_moding

In [2]:
os.environ["MODIN_ENGINE"] = "dask"

In [4]:
def open_files(paths):
    data = []
    for path in paths:
        zf = zipfile.ZipFile(path)#Función de apertura de .zip
        data.append(pd_moding.read_csv(#Función de lectura paralelizada
                        zf.open(zf.filelist[0].filename),#Abre el primer archivo del .zip
                        sep = "|",
                        encoding = "utf-8",
                        dtype=str
                        )._to_pandas()
                    )
        print(path,"ok")
    return pd.concat(data,ignore_index = True)

### Importo virtuales

In [6]:
file_paths = glob.glob("C:/Users/scadacat/Desktop/TIGO (Cliente)/Cobranzas/virtuales/*.zip")

In [7]:
virtuales = open_files(file_paths[0:5])

To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()

To request implementation, send an email to feature_requests@modin.org.


C:/Users/scadacat/Desktop/TIGO (Cliente)/Cobranzas/virtuales\Abril.zip ok
C:/Users/scadacat/Desktop/TIGO (Cliente)/Cobranzas/virtuales\Enero.zip ok
C:/Users/scadacat/Desktop/TIGO (Cliente)/Cobranzas/virtuales\febrero.zip ok
C:/Users/scadacat/Desktop/TIGO (Cliente)/Cobranzas/virtuales\Marzo.zip ok


In [8]:
vir=virtuales.copy()

In [9]:
### Está proximo a vencer su segunda factura
virtuales['# de dias de mora'][virtuales['# de dias de mora']==' '] = np.nan

In [10]:
virtuales['Descripcion De Gestion Resultado'].value_counts(True)

MENSAJE DE TEXTO ENTREGADO    0.722248
MENSAJE DE IVR ENTREGADO      0.196701
EMAIL ENTREGADO               0.081051
Name: Descripcion De Gestion Resultado, dtype: float64

In [11]:
## Cantidad de díaspara mora... eso quiere decir el el ## sea negativo
virtuales[virtuales['# de dias de mora']=='-9'].head()

,Identificación,# de suscripcion,Causal,Codigo de Gestion,# de dias de mora,Descripcion De Gestion Resultado,Calificación,Sistema Origen,Ciclo,Fecha de Gestion,Clase de Gestion
1263687,71277518,15907540,,212,-9,EMAIL ENTREGADO,,OPEN_UNE,23,2021-04-10 11:39:29-05,CORREO ELECTRONICO
1592100,800237456,7365208,,212,-9,EMAIL ENTREGADO,1,Elite,1,2021-04-19 16:58:14-05,CORREO ELECTRONICO
1592123,800237456,5977350,,212,-9,EMAIL ENTREGADO,1,Elite,1,2021-04-19 16:58:14-05,CORREO ELECTRONICO
1628379,830011670,480427,,212,-9,EMAIL ENTREGADO,0,OPEN_ETP,1,2021-04-26 12:24:48-05,CORREO ELECTRONICO
1628389,830011670,480425,,212,-9,EMAIL ENTREGADO,0,OPEN_ETP,1,2021-04-26 12:24:48-05,CORREO ELECTRONICO


In [12]:
virtuales=virtuales[~virtuales.duplicated()]

In [15]:
virtuales['# de dias de mora'][virtuales['# de dias de mora']==' '] = np.nan

In [16]:
# virtuales['# de dias de mora'].value_counts()
# virtuales['# de dias de mora'].replace("Nan", value=None).value_counts().head(20)
# virtuales['# de dias de mora']= pd.DataFrame.from_dict(
#                                             virtuales['# de dias de mora']).replace("Nan", value=None).astype(float)
# virtuales['# de dias de mora']=virtuales['# de dias de mora'].astype(float)

In [17]:
virtuales['Fecha']=virtuales['Fecha de Gestion'].str.replace('(.*)\s.*', '\\1')
virtuales['Fecha']=pd.to_datetime(virtuales['Fecha'],format='%Y-%m-%d')
virtuales['Fecha de Gestion']=pd.to_datetime(virtuales['Fecha de Gestion'],format='%Y-%m-%d %H:%M:%S')
virtuales['Ano']=virtuales['Fecha de Gestion'].dt.year
virtuales['mes']=virtuales['Fecha de Gestion'].dt.month
virtuales['hora']=virtuales['Fecha de Gestion'].dt.hour

In [18]:
virtuales['dias'] = (virtuales.sort_values(by=['Identificación','Fecha de Gestion'], ascending=[True,True])\
                .groupby(['Ano','mes','Identificación'])['Fecha'].diff())
virtuales['dias']= virtuales['dias'].astype(str)
dias = virtuales['dias'].str.split(' ',1,expand= True) ## contacots entre dias
virtuales['dias'] = dias[0]
virtuales['dias']=virtuales['dias'].str.replace('NaT','Nan')
virtuales['dias']= pd.DataFrame.from_dict(virtuales['dias']).replace("Nan", value=None).astype(float)
virtuales['dias_Class'] = pd.cut(x=virtuales['dias'], bins=[0,1, 2, 3, 7,14,21,31])
virtuales = virtuales.where(pd.notnull(virtuales), None)
# virtuales.sample(2)

In [19]:
## Cantidad de contactos por mes y grupo
virtuales_ag=virtuales.groupby(['Ano','mes','Descripcion De Gestion Resultado','Identificación']).size().\
                            reset_index(name="count")### Conteo de cantidad de contactos por grupo, año,mes y ani(Id del cliente)
virtuales_ag=virtuales_ag[virtuales_ag['count']<=100]
virtuales_ag['count2'] = pd.cut(x=virtuales_ag['count'],
                             bins=[0,1,2,3,6,10,17,1000], 
                             labels=["1","2","3","4-6","7-10", "11-17", ">17"]) 
## Realizamos una segmentacion de rangos de cantidad de contactos

In [20]:
virtuales.head(2)

,Identificación,# de suscripcion,Causal,Codigo de Gestion,# de dias de mora,Descripcion De Gestion Resultado,Calificación,Sistema Origen,Ciclo,Fecha de Gestion,Clase de Gestion,Fecha,Ano,mes,hora,dias,dias_Class
0,3483,17427156,,213,None,MENSAJE DE IVR ENTREGADO,,OPEN_UNE,23,2021-04-06 07:35:24-05:00,IVR,2021-04-06,2021,4,7,1.0,"(0, 1]"
1,3483,17427156,,213,None,MENSAJE DE IVR ENTREGADO,,OPEN_UNE,23,2021-04-29 07:35:24-05:00,IVR,2021-04-29,2021,4,7,2.0,"(1, 2]"


In [21]:
virtuales[['Ano','mes','Descripcion De Gestion Resultado','Identificación']].head(1)

,Ano,mes,Descripcion De Gestion Resultado,Identificación
0,2021,4,MENSAJE DE IVR ENTREGADO,3483


In [22]:
virtuales_ag[['Ano','mes','Descripcion De Gestion Resultado','Identificación']].head(1)

,Ano,mes,Descripcion De Gestion Resultado,Identificación
0,2021,1,EMAIL ENTREGADO,1000002457


In [23]:
vir_2=pd.merge(virtuales,virtuales_ag,on=['Ano','mes','Descripcion De Gestion Resultado','Identificación'])\
.rename(columns={'count':'cantidad','count2':'Reconta_class'})

In [24]:
vir_2.head()

,Identificación,# de suscripcion,Causal,Codigo de Gestion,# de dias de mora,Descripcion De Gestion Resultado,Calificación,Sistema Origen,Ciclo,Fecha de Gestion,Clase de Gestion,Fecha,Ano,mes,hora,dias,dias_Class,cantidad,Reconta_class
0,3483,17427156,,213,None,MENSAJE DE IVR ENTREGADO,,OPEN_UNE,23,2021-04-06 07:35:24-05:00,IVR,2021-04-06,2021,4,7,1.0,"(0, 1]",12,11-17
1,3483,17427156,,213,None,MENSAJE DE IVR ENTREGADO,,OPEN_UNE,23,2021-04-29 07:35:24-05:00,IVR,2021-04-29,2021,4,7,2.0,"(1, 2]",12,11-17
2,3483,17427156,,213,None,MENSAJE DE IVR ENTREGADO,,OPEN_UNE,23,2021-04-09 07:35:24-05:00,IVR,2021-04-09,2021,4,7,1.0,"(0, 1]",12,11-17
3,3483,17427156,,213,None,MENSAJE DE IVR ENTREGADO,,OPEN_UNE,23,2021-04-26 07:35:24-05:00,IVR,2021-04-26,2021,4,7,2.0,"(1, 2]",12,11-17
4,3483,17427156,,213,None,MENSAJE DE IVR ENTREGADO,,OPEN_UNE,23,2021-04-24 07:35:24-05:00,IVR,2021-04-24,2021,4,7,1.0,"(0, 1]",12,11-17


In [26]:
virtuales_2=pd.concat([vir_2,pd.get_dummies(vir_2['Descripcion De Gestion Resultado']),pd.get_dummies(vir_2['hora'])],axis=1) # concateno

In [28]:
grupo=virtuales_2.groupby(['Ano','mes','Identificación']).agg({'Descripcion De Gestion Resultado':'np.array',
                                        'Descripcion De Gestion Resultado':'sum',
                                        1:'sum',2:'sum',3:'sum',4:'sum',
                                        5:'sum',6:'sum',7:'sum',8:'sum',
                                        9:"sum",10:"sum",11:"sum",12:"sum",13:"sum",
                                        14:"sum",15:"sum",16:"sum",17:"sum",18:"sum",
                                        19:"sum",20:"sum",'EMAIL ENTREGADO':'sum',
                                     'MENSAJE DE IVR ENTREGADO':'sum',
                                     'MENSAJE DE TEXTO ENTREGADO':'sum'}).reset_index().\
                                        rename(columns={'MENSAJE DE IVR ENTREGADO':'MENSAJE DE IVR ENTREGADO_sum',
                                                        'MENSAJE DE TEXTO ENTREGADO':'MENSAJE DE TEXTO ENTREGADO_sum',
                                                        'EMAIL ENTREGADO':'EMAIL ENTREGADO_sum',
                                                        'Descripcion De Gestion Resultado':'Descripcion De Gestion Resultado_sum'})

In [29]:
grupo.head()

,Ano,mes,Identificación,Descripcion De Gestion Resultado_sum,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,EMAIL ENTREGADO_sum,MENSAJE DE IVR ENTREGADO_sum,MENSAJE DE TEXTO ENTREGADO_sum
0,2021,1,1000002457,MENSAJE DE TEXTO ENTREGADOMENSAJE DE TEXTO ENT...,0,0,0,0,0,0,1,0,0,3,0,3,1,0,0,0,0,0,0,0,3,1,4
1,2021,1,1000006765,MENSAJE DE IVR ENTREGADOMENSAJE DE IVR ENTREGA...,0,0,0,0,0,0,2,0,0,3,0,3,1,0,0,0,0,0,0,0,3,2,4
2,2021,1,1000018742,MENSAJE DE TEXTO ENTREGADOMENSAJE DE TEXTO ENT...,0,0,0,0,0,0,0,0,0,4,1,0,3,0,1,1,0,0,0,0,2,0,8
3,2021,1,1000036872,MENSAJE DE TEXTO ENTREGADOMENSAJE DE TEXTO ENT...,0,0,0,0,0,0,0,0,0,4,1,2,1,0,0,0,0,0,0,0,3,0,5
4,2021,1,1000047106,EMAIL ENTREGADOMENSAJE DE TEXTO ENTREGADOMENSA...,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,2


In [30]:
print(grupo.columns.to_list())

['Ano', 'mes', 'Identificación', 'Descripcion De Gestion Resultado_sum', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 'EMAIL ENTREGADO_sum', 'MENSAJE DE IVR ENTREGADO_sum', 'MENSAJE DE TEXTO ENTREGADO_sum']


In [31]:
consolidado=pd.merge(vir_2,grupo[['Identificación', 'Descripcion De Gestion Resultado_sum',
                                  1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
                                  'EMAIL ENTREGADO_sum', 'MENSAJE DE IVR ENTREGADO_sum',
                                  'MENSAJE DE TEXTO ENTREGADO_sum']],on='Identificación')

In [33]:
consolidado=consolidado.sort_values(['Fecha de Gestion','Identificación','dias'],
                                    ascending=[True,True,True]).drop_duplicates('Identificación',keep='last')

In [37]:
# consolidado=consolidado.sort_values(['Fecha de Gestion','Identificación','dias'],
#                                     ascending=[True,True,True]).drop_duplicates('Identificación',keep='last')

In [35]:
consolidado.drop(['Descripcion De Gestion Resultado','Descripcion De Gestion Resultado_sum','Causal','Clase de Gestion'],axis=1,inplace=True)

In [36]:
consolidado.tail()

,Identificación,# de suscripcion,Codigo de Gestion,# de dias de mora,Calificación,Sistema Origen,Ciclo,Fecha de Gestion,Fecha,Ano,mes,hora,dias,dias_Class,cantidad,Reconta_class,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,EMAIL ENTREGADO_sum,MENSAJE DE IVR ENTREGADO_sum,MENSAJE DE TEXTO ENTREGADO_sum
192793,9915061,1055974,212,None,,OPEN_ETP,1,2021-04-30 10:41:33-05:00,2021-04-30,2021,4,10,1.0,"(0.0, 1.0]",24,>17,0,0,0,0,0,0,2,0,0,26,0,10,2,4,0,0,0,0,0,0,24,2,18
1679159,99555011,16067622,212,None,,OPEN_UNE,40,2021-04-30 10:41:33-05:00,2021-04-30,2021,4,10,1.0,"(0.0, 1.0]",1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,1
1785234,830131993,113869,212,0,0,OPEN_ETP,1,2021-04-30 13:53:42-05:00,2021-04-30,2021,4,13,17.0,"(14.0, 21.0]",17,11-17,0,0,0,0,0,0,0,0,0,5,0,0,12,0,0,0,0,0,0,0,17,0,0
1821206,890919291,10047162,212,-19,0.95,OPEN_UNE,36,2021-04-30 14:16:25-05:00,2021-04-30,2021,4,14,11.0,"(7.0, 14.0]",8,7-10,0,0,0,0,0,0,0,0,0,0,0,4,0,4,0,0,0,0,0,0,8,0,0
1704939,800104062,444202,212,0,1,OPEN_EDATEL,1,2021-04-30 16:25:38-05:00,2021-04-30,2021,4,16,0.0,NaN,96,>17,0,0,0,0,0,0,0,0,0,0,0,30,32,0,0,34,0,0,0,0,96,0,0


## Exporto para agilizar el proceso

In [38]:
consolidado.to_csv('C:/Users/scadacat/Desktop/TIGO (Cliente)/Cobranzas/Exportes de Bd_cobranzas/virtuales_2.csv',encoding='utf-8',sep=';',index=False)

## Fin del notebook